In [15]:
print("Hello World!")

Hello World!


In [ ]:
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly

In [3]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
import plotly.io as pio
pio.renderers.default = "iframe"



In [5]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [6]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    from IPython.display import display, HTML
    fig_html = fig.to_html()
    display(HTML(fig_html))

In [7]:
#Question 1: Use yfinance to Extract Stock Data
tesla = yf.Ticker('TSLA')
tesla_data = tesla.history(period="max")

In [8]:
#Question 2: Use Webscraping to Extract Tesla Revenue Data
url ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text
soup_x = BeautifulSoup(html_data,"html.parser")
tesla_revenue = pd.DataFrame(columns=["Date","Revenue"])
for row in soup_x.find("tbody").find_all("tr"):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text
    #append to dataframe
    tesla_revenue = pd.concat([tesla_revenue,pd.DataFrame({"Date" :[date], "Revenue":[revenue]})])
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"",regex=True)
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]
print(tesla_revenue.tail(5))

   Date Revenue
0  2013    2013
0  2012     413
0  2011     204
0  2010     117
0  2009     112


In [9]:
#Question 3: Use yfinance to Extract Stock Data
#Making Ticker Object
GameStop = yf.Ticker('GME')
#set the ticker object using period and max
gme_data = GameStop.history(period="max")
#tesla_data.reset_index(inplace=True)
gme_data.reset_index(inplace=True)
print(gme_data.head(5))

                       Date      Open      High       Low     Close    Volume  \
0 2002-02-13 00:00:00-05:00  1.620128  1.693350  1.603296  1.691666  76216000   
1 2002-02-14 00:00:00-05:00  1.712707  1.716074  1.670626  1.683251  11021600   
2 2002-02-15 00:00:00-05:00  1.683250  1.687458  1.658001  1.674834   8389600   
3 2002-02-19 00:00:00-05:00  1.666418  1.666418  1.578047  1.607504   7410400   
4 2002-02-20 00:00:00-05:00  1.615920  1.662210  1.603296  1.662210   6892800   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  


In [10]:
#Question 4: Use Webscraping to Extract GME Revenue Data
html_data_2 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
soup_3 = requests.get(html_data_2).text
beautiful_soup = BeautifulSoup(soup_3, "html.parser")
!pip install lxml
#Extract the table
GameStop_Revenue = pd.read_html(html_data_2)

#convert BeautifulSoap to string
GameStop_Revenue = pd.read_html(str(soup_3))
#To make dataframe whose "Date" & "Revenue" columns
gme_revenue = pd.DataFrame(columns = ["Date", "Revenue"])
for row_1 in beautiful_soup.find("tbody").find_all("tr"):
    col_1 = row_1.find_all("td")
    date_1 = col_1[0].text
    revenue_1 = col_1[1].text
    #append to daaframe
    gme_revenue = pd.concat([gme_revenue,pd.DataFrame({"Date" :[date_1], "Revenue" :[revenue_1]})])

# Remove comma & dollar from Revenue column
gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace(',|\$',"",regex=True)
print(gme_revenue.tail(5))

   Date Revenue
0  2009    8806
0  2008    7094
0  2007    5319
0  2006    3092
0  2005    1843


In [12]:
#Question 5: Plot Tesla Stock Graph
from datetime import datetime
# cutoff date
cutoff_date = datetime(2021, 6, 1)

def make_graph(tesla_revenue):
    # filter data sebelum Juni 2021
    revenue_before_june_2021 = tesla_revenue[tesla_revenue['Date'] < cutoff_date]

    # Buat grafik
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=revenue_before_june_2021['Date'],
        y=revenue_before_june_2021['Revenue'],
        mode='lines',
        name='Revenue'
    ))

    # Customisasi layout
    fig.update_layout(
        title="Graph of Revenue",
        xaxis_title="Date",
        yaxis_title="Revenue",
        xaxis=dict(
            tickmode='array',
            tickvals=revenue_before_june_2021['Date'],
        )
    )

    # tampilkan grafik
    fig.show()

    print(f"Creating a line graph with the title: Graph of Revenue")

In [13]:
#Question 6: Plot GameStop Stock Graph
# cutoff date
cutoff_date = datetime(2021, 6, 1)

def make_graph(gme_revenue):
    # filter data sebelum Juni 2021
    revenue_before_june_2021 = gme_revenue[gme_revenue['Date'] < cutoff_date]

    # Buat grafik
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=revenue_before_june_2021['Date'],
        y=revenue_before_june_2021['Revenue'],
        mode='lines',
        name='Revenue'
    ))

    # Customisasi layout
    fig.update_layout(
        title="Graph of Revenue",
        xaxis_title="Date",
        yaxis_title="Revenue",
        xaxis=dict(
            tickmode='array',
            tickvals=revenue_before_june_2021['Date'],
        )
    )

    # tampilkan grafik
    fig.show()

    print(f"Creating a line graph with the title: Graph of Revenue")